# Install the library (run this cell if using Colab or if you haven't installed the package)
!pip install --upgrade --no-cache-dir simple-backtest yfinance

In [ ]:
# Install the library (run this cell if using Colab or if you haven't installed the package)
!pip install simple-backtest yfinance

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from typing import Any, Dict, List

from simple_backtest import BacktestConfig, Backtest
from simple_backtest.strategy import Strategy
from simple_backtest.visualization import plot_equity_curve

## Load Data

In [2]:
# Download data - using QQQ (Nasdaq ETF) for good volatility
ticker = "QQQ"
data = yf.download(ticker, start="2020-01-01", end="2023-12-31", progress=False)
data = data.dropna()

print(f"Data shape: {data.shape}")
print(f"Date range: {data.index[0]} to {data.index[-1]}")
data.tail()

/var/folders/9y/pp82h2912h3781myzdncvs_r0000gp/T/ipykernel_55120/3148259390.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2020-01-01", end="2023-12-31", progress=False)


Data shape: (1006, 5)
Date range: 2020-01-02 00:00:00 to 2023-12-29 00:00:00


Price,Close,High,Low,Open,Volume
Ticker,QQQ,QQQ,QQQ,QQQ,QQQ
Date,,,,,
2023-12-22,404.226715,405.800541,402.346044,404.840405,34314000
2023-12-26,406.701355,407.374432,404.988938,405.087927,22722500
2023-12-27,407.529266,407.816476,406.122955,406.984586,31980500
2023-12-28,407.331207,408.935601,406.974696,408.688013,27029200
2023-12-29,405.568359,407.667928,403.647077,407.311386,42662100


# Download data - using QQQ (Nasdaq ETF) for good volatility
ticker = "QQQ"
data = yf.download(ticker, start="2020-01-01", end="2023-12-31", progress=False)

# Handle MultiIndex columns if present
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)

data = data.dropna()

print(f"Data shape: {data.shape}")
print(f"Date range: {data.index[0]} to {data.index[-1]}")
data.tail()

In [3]:
class BollingerBandsStrategy(Strategy):
    """Mean reversion strategy using Bollinger Bands."""
    
    def __init__(self, period: int = 20, num_std: float = 2.0, shares: float = 10, name: str = None):
        super().__init__(name=name or f"BB_{period}_{num_std}")
        self.period = period
        self.num_std = num_std
        self.shares = shares
    
    def calculate_bands(self, data: pd.DataFrame):
        """Calculate Bollinger Bands."""
        prices = data['Close']
        
        # Middle band (SMA)
        middle = prices.tail(self.period).mean()
        
        # Standard deviation
        std = prices.tail(self.period).std()
        
        # Upper and lower bands
        upper = middle + (std * self.num_std)
        lower = middle - (std * self.num_std)
        
        return lower, middle, upper
    
    def predict(self, data: pd.DataFrame, trade_history: List[Dict[str, Any]]) -> Dict[str, Any]:
        if len(data) < self.period:
            return self.hold()
        
        lower, middle, upper = self.calculate_bands(data)
        current_price = data['Close'].iloc[-1]
        
        # Buy when price touches lower band (oversold)
        if current_price <= lower and not self.has_position():
            return self.buy(self.shares)
        
        # Sell when price touches upper band (overbought)
        if current_price >= upper and self.has_position():
            return self.sell(self.shares)
        
        # Exit at middle band (mean reversion complete)
        if self.has_position() and abs(current_price - middle) < 0.01 * middle:
            return self.sell(self.shares)
        
        return self.hold()

print("✓ BollingerBandsStrategy created")

✓ BollingerBandsStrategy created


## 2. RSI Strategy

**Relative Strength Index (RSI)** measures momentum:
- Range: 0 to 100
- **Overbought**: RSI > 70
- **Oversold**: RSI < 30

**Strategy**:
- **Buy** when RSI crosses above 30 (oversold reversal)
- **Sell** when RSI crosses below 70 (overbought reversal)

In [4]:
class RSIStrategy(Strategy):
    """Momentum strategy using RSI indicator."""
    
    def __init__(self, period: int = 14, oversold: float = 30, overbought: float = 70, 
                 shares: float = 10, name: str = None):
        super().__init__(name=name or f"RSI_{period}")
        self.period = period
        self.oversold = oversold
        self.overbought = overbought
        self.shares = shares
    
    def calculate_rsi(self, data: pd.DataFrame):
        """Calculate RSI indicator."""
        prices = data['Close']
        
        if len(prices) < self.period + 1:
            return 50  # Neutral RSI
        
        # Calculate price changes
        deltas = prices.diff()
        
        # Separate gains and losses
        gains = deltas.where(deltas > 0, 0)
        losses = -deltas.where(deltas < 0, 0)
        
        # Calculate average gains and losses
        avg_gain = gains.tail(self.period).mean()
        avg_loss = losses.tail(self.period).mean()
        
        if avg_loss == 0:
            return 100
        
        # Calculate RS and RSI
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        
        return rsi
    
    def predict(self, data: pd.DataFrame, trade_history: List[Dict[str, Any]]) -> Dict[str, Any]:
        if len(data) < self.period + 1:
            return self.hold()
        
        rsi = self.calculate_rsi(data)
        
        # Buy when RSI is oversold
        if rsi < self.oversold and not self.has_position():
            return self.buy(self.shares)
        
        # Sell when RSI is overbought
        if rsi > self.overbought and self.has_position():
            return self.sell(self.shares)
        
        return self.hold()

print("✓ RSIStrategy created")

✓ RSIStrategy created


## 3. MACD Strategy

**MACD** consists of:
- **MACD Line**: 12-day EMA - 26-day EMA
- **Signal Line**: 9-day EMA of MACD line
- **Histogram**: MACD line - Signal line

**Strategy**:
- **Buy** when MACD crosses above signal line (bullish crossover)
- **Sell** when MACD crosses below signal line (bearish crossover)

In [5]:
class MACDStrategy(Strategy):
    """Trend-following strategy using MACD indicator."""
    
    def __init__(self, fast: int = 12, slow: int = 26, signal: int = 9, 
                 shares: float = 10, name: str = None):
        super().__init__(name=name or f"MACD_{fast}_{slow}_{signal}")
        self.fast = fast
        self.slow = slow
        self.signal = signal
        self.shares = shares
        self.prev_macd = None
        self.prev_signal_line = None
    
    def calculate_ema(self, prices: pd.Series, period: int):
        """Calculate Exponential Moving Average."""
        return prices.tail(period).ewm(span=period, adjust=False).mean().iloc[-1]
    
    def calculate_macd(self, data: pd.DataFrame):
        """Calculate MACD and signal line."""
        prices = data['Close']
        
        if len(prices) < self.slow:
            return 0, 0
        
        # Calculate EMAs
        ema_fast = self.calculate_ema(prices, self.fast)
        ema_slow = self.calculate_ema(prices, self.slow)
        
        # MACD line
        macd = ema_fast - ema_slow
        
        # For signal line, we need MACD history
        if len(prices) < self.slow + self.signal:
            signal_line = 0
        else:
            # Approximate signal line using recent MACD values
            # In production, you'd maintain MACD history
            signal_line = macd * 0.9  # Simplified approximation
        
        return macd, signal_line
    
    def predict(self, data: pd.DataFrame, trade_history: List[Dict[str, Any]]) -> Dict[str, Any]:
        if len(data) < self.slow + self.signal:
            return self.hold()
        
        macd, signal_line = self.calculate_macd(data)
        
        # Detect crossovers
        if self.prev_macd is not None and self.prev_signal_line is not None:
            # Bullish crossover: MACD crosses above signal
            bullish_cross = (self.prev_macd <= self.prev_signal_line) and (macd > signal_line)
            
            # Bearish crossover: MACD crosses below signal
            bearish_cross = (self.prev_macd >= self.prev_signal_line) and (macd < signal_line)
            
            if bullish_cross and not self.has_position():
                self.prev_macd = macd
                self.prev_signal_line = signal_line
                return self.buy(self.shares)
            
            if bearish_cross and self.has_position():
                self.prev_macd = macd
                self.prev_signal_line = signal_line
                return self.sell(self.shares)
        
        # Store current values for next iteration
        self.prev_macd = macd
        self.prev_signal_line = signal_line
        
        return self.hold()
    
    def reset_state(self):
        super().reset_state()
        self.prev_macd = None
        self.prev_signal_line = None

print("✓ MACDStrategy created")

✓ MACDStrategy created


## 4. Stochastic Oscillator Strategy

**Stochastic Oscillator** compares closing price to price range:
- **%K**: Current momentum
- **%D**: 3-period SMA of %K (signal line)
- Range: 0 to 100

**Strategy**:
- **Buy** when %K crosses above %D in oversold zone (< 20)
- **Sell** when %K crosses below %D in overbought zone (> 80)

In [6]:
class StochasticStrategy(Strategy):
    """Momentum strategy using Stochastic Oscillator."""
    
    def __init__(self, k_period: int = 14, d_period: int = 3, oversold: float = 20,
                 overbought: float = 80, shares: float = 10, name: str = None):
        super().__init__(name=name or f"Stochastic_{k_period}_{d_period}")
        self.k_period = k_period
        self.d_period = d_period
        self.oversold = oversold
        self.overbought = overbought
        self.shares = shares
        self.prev_k = None
        self.prev_d = None
    
    def calculate_stochastic(self, data: pd.DataFrame):
        """Calculate Stochastic Oscillator %K and %D."""
        if len(data) < self.k_period:
            return 50, 50
        
        # Get recent data
        recent = data.tail(self.k_period)
        
        # Calculate %K
        high = recent['High'].max()
        low = recent['Low'].min()
        close = recent['Close'].iloc[-1]
        
        if high == low:
            k = 50
        else:
            k = 100 * (close - low) / (high - low)
        
        # Calculate %D (simplified - using previous K values approximation)
        if len(data) < self.k_period + self.d_period:
            d = k
        else:
            d = k * 0.7 + (self.prev_k if self.prev_k else k) * 0.3  # Simplified
        
        return k, d
    
    def predict(self, data: pd.DataFrame, trade_history: List[Dict[str, Any]]) -> Dict[str, Any]:
        if len(data) < self.k_period:
            return self.hold()
        
        k, d = self.calculate_stochastic(data)
        
        # Detect crossovers in oversold/overbought zones
        if self.prev_k is not None and self.prev_d is not None:
            # Bullish: %K crosses above %D in oversold zone
            bullish = (self.prev_k <= self.prev_d) and (k > d) and (k < self.oversold)
            
            # Bearish: %K crosses below %D in overbought zone
            bearish = (self.prev_k >= self.prev_d) and (k < d) and (k > self.overbought)
            
            if bullish and not self.has_position():
                self.prev_k = k
                self.prev_d = d
                return self.buy(self.shares)
            
            if bearish and self.has_position():
                self.prev_k = k
                self.prev_d = d
                return self.sell(self.shares)
        
        self.prev_k = k
        self.prev_d = d
        
        return self.hold()
    
    def reset_state(self):
        super().reset_state()
        self.prev_k = None
        self.prev_d = None

print("✓ StochasticStrategy created")

✓ StochasticStrategy created


## 5. Volume Strategy (OBV)

**On-Balance Volume (OBV)** relates volume to price changes:
- Add volume on up days
- Subtract volume on down days
- OBV trend confirms price trend

**Strategy**:
- **Buy** when OBV is rising and crosses above its MA (accumulation)
- **Sell** when OBV is falling and crosses below its MA (distribution)

In [7]:
class OBVStrategy(Strategy):
    """Volume-based strategy using On-Balance Volume."""
    
    def __init__(self, ma_period: int = 20, shares: float = 10, name: str = None):
        super().__init__(name=name or f"OBV_{ma_period}")
        self.ma_period = ma_period
        self.shares = shares
    
    def calculate_obv(self, data: pd.DataFrame):
        """Calculate On-Balance Volume."""
        if len(data) < 2:
            return 0
        
        # Calculate OBV cumulatively
        obv = 0
        for i in range(1, len(data)):
            if data['Close'].iloc[i] > data['Close'].iloc[i-1]:
                obv += data['Volume'].iloc[i]
            elif data['Close'].iloc[i] < data['Close'].iloc[i-1]:
                obv -= data['Volume'].iloc[i]
            # If close unchanged, OBV unchanged
        
        return obv
    
    def predict(self, data: pd.DataFrame, trade_history: List[Dict[str, Any]]) -> Dict[str, Any]:
        if len(data) < self.ma_period + 1:
            return self.hold()
        
        # Calculate current OBV
        current_obv = self.calculate_obv(data)
        
        # Calculate OBV for previous window (for MA)
        prev_obvs = []
        for i in range(len(data) - self.ma_period, len(data)):
            window_obv = self.calculate_obv(data.iloc[:i+1])
            prev_obvs.append(window_obv)
        
        obv_ma = np.mean(prev_obvs) if prev_obvs else current_obv
        
        # Price trend
        price_rising = data['Close'].iloc[-1] > data['Close'].iloc[-5]
        
        # OBV above MA and price rising = accumulation (buy)
        if current_obv > obv_ma and price_rising and not self.has_position():
            return self.buy(self.shares)
        
        # OBV below MA and price not rising = distribution (sell)
        if current_obv < obv_ma and not price_rising and self.has_position():
            return self.sell(self.shares)
        
        return self.hold()

print("✓ OBVStrategy created")

✓ OBVStrategy created


## Compare All Technical Indicator Strategies

In [8]:
# Configure backtest
config = BacktestConfig(
    initial_capital=10000.0,
    lookback_period=50,  # Longer lookback for TA indicators
    commission_type="percentage",
    commission_value=0.001,
    risk_free_rate=0.02
)

# Create all TA strategies
strategies = [
    BollingerBandsStrategy(period=20, num_std=2.0, shares=10),
    RSIStrategy(period=14, oversold=30, overbought=70, shares=10),
    MACDStrategy(fast=12, slow=26, signal=9, shares=10),
    StochasticStrategy(k_period=14, d_period=3, shares=10),
    OBVStrategy(ma_period=20, shares=10),
]

print("Technical Analysis Strategies:")
for s in strategies:
    print(f"  - {s.get_name()}")

Technical Analysis Strategies:
  - BB_20_2.0
  - RSI_14
  - MACD_12_26_9
  - Stochastic_14_3
  - OBV_20


In [9]:
# Run backtest
backtest = Backtest(data=data, config=config)
results = backtest.run(strategies)

print("\nBacktest completed!")


Backtest completed!


In [10]:
# Compare results
comparison_df = results.compare()

print("\nTechnical Analysis Strategy Comparison:")
print("=" * 120)
display_cols = ['total_return', 'cagr', 'sharpe_ratio', 'sortino_ratio', 'max_drawdown', 
                'volatility', 'total_trades', 'win_rate', 'profit_factor']
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print(comparison_df[display_cols])


Technical Analysis Strategy Comparison:
                 total_return       cagr  sharpe_ratio  sortino_ratio  \
benchmark          141.816862  26.263761      0.979825       1.373043   
OBV_20              17.691835   4.396056      0.567085       0.763645   
MACD_12_26_9        12.163093   3.077854      0.244420       0.310360   
BB_20_2.0           11.737407   2.974392      0.269252       0.412901   
Stochastic_14_3     11.302089   2.868289      0.194289       0.264881   
RSI_14               8.626166   2.209269      0.066044       0.090438   

                 max_drawdown  volatility  total_trades   win_rate  \
benchmark           36.691468   24.982367           1.0   0.000000   
OBV_20               5.911783    4.247469          95.0  48.936170   
MACD_12_26_9         8.089834    4.752512          43.0  52.380952   
BB_20_2.0            4.266898    3.787958          40.0  70.000000   
Stochastic_14_3      7.647770    4.991731          32.0  81.250000   
RSI_14               8.5882

In [11]:
# Best strategies
print("\n" + "="*70)
print("BEST STRATEGIES BY DIFFERENT METRICS")
print("="*70)

metrics_to_check = ['sharpe_ratio', 'total_return', 'sortino_ratio', 'win_rate']
for metric in metrics_to_check:
    best = results.best_strategy(metric)  # Returns StrategyResult object
    value = best.metrics[metric]  # Access metrics directly from StrategyResult
    if 'return' in metric or 'drawdown' in metric or 'rate' in metric:
        print(f"\n{metric.upper()}: {best.name}")
        print(f"  Value: {value*100:.2f}%")
    else:
        print(f"\n{metric.upper()}: {best.name}")
        print(f"  Value: {value:.2f}")


BEST STRATEGIES BY DIFFERENT METRICS

SHARPE_RATIO: OBV_20
  Value: 0.57

TOTAL_RETURN: OBV_20
  Value: 1769.18%

SORTINO_RATIO: OBV_20
  Value: 0.76

WIN_RATE: Stochastic_14_3
  Value: 8125.00%


In [12]:
# Visualize comparison
fig = results.plot_comparison()
fig.update_layout(title="Technical Analysis Strategies - Equity Curves")
fig.show()

## Deep Dive: Bollinger Bands Strategy

In [13]:
# Detailed analysis of Bollinger Bands strategy
bb_result = results.get_strategy('BB_20_2.0')
metrics = bb_result.metrics

print("\n" + "="*80)
print("BOLLINGER BANDS STRATEGY - DETAILED ANALYSIS")
print("="*80)

print(f"\n📊 Configuration:")
print(f"  Period: 20 days")
print(f"  Standard Deviations: 2.0")
print(f"  Shares per trade: 10")

print(f"\n💰 Performance:")
print(f"  Total Return:        {metrics['total_return']*100:>10.2f}%")
print(f"  CAGR:                {metrics['cagr']*100:>10.2f}%")
print(f"  Sharpe Ratio:        {metrics['sharpe_ratio']:>10.2f}")
print(f"  Sortino Ratio:       {metrics['sortino_ratio']:>10.2f}")
print(f"  Calmar Ratio:        {metrics['calmar_ratio']:>10.2f}")

print(f"\n⚠️  Risk:")
print(f"  Max Drawdown:        {metrics['max_drawdown']*100:>10.2f}%")
print(f"  Volatility:          {metrics['volatility']*100:>10.2f}%")

print(f"\n📈 Trading:")
print(f"  Total Trades:        {metrics['total_trades']:>10}")
print(f"  Win Rate:            {metrics['win_rate']*100:>10.2f}%")
print(f"  Profit Factor:       {metrics['profit_factor']:>10.2f}")
print(f"  Avg Win:             ${metrics['avg_win']:>10.2f}")
print(f"  Avg Loss:            ${metrics['avg_loss']:>10.2f}")
print(f"  Expectancy:          ${metrics['expectancy']:>10.2f}")

print(f"\n🎯 vs Benchmark:")
print(f"  Alpha:               {metrics['alpha']*100:>10.2f}%")
print(f"  Beta:                {metrics['beta']:>10.2f}")
print(f"  Information Ratio:   {metrics['information_ratio']:>10.2f}")

print("\n" + "="*80)


BOLLINGER BANDS STRATEGY - DETAILED ANALYSIS

📊 Configuration:
  Period: 20 days
  Standard Deviations: 2.0
  Shares per trade: 10

💰 Performance:
  Total Return:           1173.74%
  CAGR:                    297.44%
  Sharpe Ratio:              0.27
  Sortino Ratio:             0.41
  Calmar Ratio:              0.70

⚠️  Risk:
  Max Drawdown:            426.69%
  Volatility:              378.80%

📈 Trading:
  Total Trades:                40
  Win Rate:               7000.00%
  Profit Factor:             5.10
  Avg Win:             $    109.89
  Avg Loss:            $    -50.32
  Expectancy:          $     61.83

🎯 vs Benchmark:
  Alpha:                    81.47%
  Beta:                      0.08
  Information Ratio:        -1.01



## Summary

In this notebook, we explored technical analysis strategies:

1. ✅ **Bollinger Bands**: Mean reversion based on volatility bands
2. ✅ **RSI**: Momentum strategy using overbought/oversold levels
3. ✅ **MACD**: Trend-following using moving average convergence/divergence
4. ✅ **Stochastic Oscillator**: Momentum indicator comparing close to range
5. ✅ **OBV**: Volume-based strategy tracking accumulation/distribution

### Key Insights:

- **Mean Reversion vs Trend Following**: Bollinger Bands and RSI are mean reversion, while MACD follows trends
- **Indicator Combinations**: Better results when combining indicators (e.g., RSI + MACD)
- **Parameter Sensitivity**: All indicators need parameter tuning for optimal performance
- **Market Conditions**: Different indicators work better in different market conditions
  - Bollinger Bands: Range-bound markets
  - MACD: Trending markets
  - RSI/Stochastic: Both trending and ranging

### Best Practices:

1. **Confirm Signals**: Use multiple indicators to confirm signals
2. **Timeframe**: Match indicator period to trading timeframe
3. **Divergence**: Watch for price-indicator divergence (strong signal)
4. **Adaptive**: Adjust parameters based on market volatility
5. **Risk Management**: Use stop-losses even with strong signals

### Next Steps:

- Combine multiple TA indicators in one strategy
- Add machine learning to optimize indicator parameters
- Test on different timeframes (intraday, weekly)
- Create adaptive strategies that adjust to market conditions